In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [11]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [13]:
df_bbidding.Target_MW

0        530.112
1        529.278
2        530.603
3        530.965
4        530.674
          ...   
42280     23.648
42281     10.865
42282     10.505
42283      4.486
42284      6.580
Name: Target_MW, Length: 42285, dtype: float64

In [12]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [14]:
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))

In [16]:
df_bbidding.Revenue_50.mean()

25437.310022730526

50% quantil gibt historisch 25437.310022730526 pro periode

In [18]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))

# Display the updated DataFrame with optimized trades and corresponding revenues
print(df_bbidding)


                   timestamp_utc  Mean_SolarRadiation_dwd  \
0      2021-02-19 23:00:00+00:00                 0.000000   
1      2021-02-19 23:30:00+00:00                 0.000000   
2      2021-02-21 00:00:00+00:00                 0.000000   
3      2021-02-21 00:30:00+00:00                 0.000000   
4      2021-02-21 01:00:00+00:00                 0.000000   
...                          ...                      ...   
42280  2023-08-25 23:30:00+00:00                 0.009818   
42281  2023-08-26 00:00:00+00:00                 0.021356   
42282  2023-08-26 00:30:00+00:00                 0.012875   
42283  2023-08-26 01:00:00+00:00                 0.004395   
42284  2023-08-26 01:30:00+00:00                 0.025069   

       SolarDownwardRadiation_RW_dwd_Mean_30min  \
0                                      0.000000   
1                                      0.000000   
2                                      0.000000   
3                                      0.000000   
4           

Mit optimierung 56488.5046925057 pro periode

In [20]:
df_bbidding.revenue.mean()

56488.5046925057

In [ ]:
𝑅𝑒𝑣𝑒𝑛𝑢𝑒=𝑇𝑟𝑎𝑑𝑒×𝐷𝐴𝑃+(𝐴𝑐𝑡𝑢𝑎𝑙− 𝑇𝑟𝑎𝑑𝑒)×(𝑆𝑆𝑃−0.07×(𝐴𝑐𝑡𝑢𝑎𝑙− 𝑇𝑟𝑎𝑑𝑒))

Quantile

In [8]:
def Value_calculator(Real_value,bid,imbalance,day_ahead):
    Revenue = bid*day_ahead +(Real_value-bid)*imbalance
    return np.mean(Revenue)

In [11]:
quantiles = ["quantile_0","quantile_1","quantile_2","quantile_3","quantile_4","quantile_5","quantile_6","quantile_7","quantile_8"]
Revenue = []
for i in quantiles:
    Revenue.append(Value_calculator(result["Solar_MWh_credit"],result[i],result["imbalance_price"],result["price"]))

In [12]:
Revenue

[17678.065502705973,
 17761.467307058454,
 17818.87789210751,
 17864.654886340366,
 17906.182712271337,
 17947.770259132223,
 17992.756566419466,
 18045.685351896656,
 18129.739720575395]

monte carlo: 

In [ ]:
import numpy as np

# Define revenue calculation function
def calculate_revenue(trade, actual, DAP, SSP):
    return trade * DAP + (actual - trade) * (SSP - 0.07 * (actual - trade))

# Monte Carlo simulation to estimate expected revenue
def monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP, num_samples=1000):
    expected_revenue = []
    
    # Sample from the probabilistic distribution (Monte Carlo simulation)
    sampled_actuals = actual_distribution(num_samples)
    
    for trade in trade_values:
        # Calculate revenue for each sample and trade value
        revenues = [calculate_revenue(trade, actual, DAP, SSP) for actual in sampled_actuals]
        # Store the average expected revenue for this trade value
        expected_revenue.append(np.mean(revenues))
    
    return trade_values, expected_revenue

# Example usage
# Define input values (these would be dynamic in practice)
DAP = 50  # Day-Ahead Price
SSP = 45  # System Sell Price

# Generate probabilistic forecast (e.g., Normal distribution with mean and std dev from your forecast)
actual_mean = 100  # Assume mean production is 100 MWh
actual_std = 10    # Assume a standard deviation of 10 MWh
actual_distribution = lambda n: np.random.normal(actual_mean, actual_std, n)

# Define the range of Trade values to evaluate
trade_values = np.linspace(80, 120, 50)  # Evaluate Trade between 80 and 120 MWh

# Run Monte Carlo simulation
trade_values, expected_revenues = monte_carlo_simulation(trade_values, actual_distribution, DAP, SSP)

# Find the optimal trade value that maximizes expected revenue
optimal_trade = trade_values[np.argmax(expected_revenues)]

print(f"Optimal Trade Value: {optimal_trade} MWh")


1. Dynamic Programming Approach


In [ ]:
for t in range(T-1, -1, -1):  # Start at last time step, move backwards
    for s in state_space:  # For each state in the state space
        max_value = -inf
        for a in action_space:  # For each possible trade action
            next_state = transition_function(s, a)
            immediate_reward = revenue(s, a)
            future_value = gamma * V[next_state]  # Discounted future value
            total_value = immediate_reward + future_value
            if total_value > max_value:
                max_value = total_value
                best_action[s] = a  # Store the best action for state s
        V[s] = max_value
